In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [2]:
from ktrain import text
import os

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/Resistance_plus_batch_1_batch_2_merged.xlsx"
resistance_df = pd.read_excel(file_path)


In [5]:
# Perform train-test split (keeping stratification)
train_resistance, test_resistance = train_test_split(
    resistance_df, test_size=0.22, random_state=42, stratify=resistance_df["label"]
)

# Verify counts
print(f"resistance Train: {len(train_resistance)}, Test: {len(test_resistance)}, Total: {len(resistance_df)}")

# Count total samples
total_train = len(train_resistance)
total_test = len(test_resistance)

# Count positive (Class 1) samples
train_positives = train_resistance["label"].sum()
test_positives = test_resistance["label"].sum()

# Calculate percentages
train_positive_pct = (train_positives / total_train) * 100
test_positive_pct = (test_positives / total_test) * 100

# Print results
print(f"📊 resistance Dataset Train-Test Split Results:")
print(f"✅ Train: {total_train} samples (Positives: {train_positives}, {train_positive_pct:.2f}%)")
print(f"✅ Test: {total_test} samples (Positives: {test_positives}, {test_positive_pct:.2f}%)")
print(f"🔹 Total Dataset: {len(resistance_df)} samples")


# Define save directory
save_dir = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/res_train_test"
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Save train and test sets
train_resistance.to_excel(f"{save_dir}/train_resistance.xlsx", index=False)
test_resistance.to_excel(f"{save_dir}/test_resistance.xlsx", index=False)

print("✅ resistance train and test sets successfully created and saved!")

resistance Train: 2208, Test: 624, Total: 2832
📊 resistance Dataset Train-Test Split Results:
✅ Train: 2208 samples (Positives: 652, 29.53%)
✅ Test: 624 samples (Positives: 184, 29.49%)
🔹 Total Dataset: 2832 samples
✅ resistance train and test sets successfully created and saved!


In [6]:
MAXLEN = 150

In [7]:
X = train_resistance['sentence']
y = train_resistance['label']

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [10]:
model_name = 'microsoft/deberta-v3-base'

In [11]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 41
	99percentile : 57


/Users/gbaldonado/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 63


In [12]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [ ]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
295/295 [==============================] - 1041s 3s/step - loss: 0.6260 - accuracy: 0.6716 - val_loss: 0.6011 - val_accuracy: 0.7036
Epoch 2/5
 25/295 [=>............................] - ETA: 23:16 - loss: 0.5951 - accuracy: 0.7133

In [17]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

13/13 [==============================] - 6s 262ms/step
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       306
           1       0.77      0.76      0.76        87

    accuracy                           0.90       393
   macro avg       0.85      0.85      0.85       393
weighted avg       0.90      0.90      0.90       393



array([[286,  20],
       [ 21,  66]])

In [19]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [21]:
distillbert_test_data = test_resistance['sentence'].tolist()
distillbert_test_label = test_resistance['label'].tolist()

In [22]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [23]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [24]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 411, False Positive: 21, False Negative: 19, True Positive: 104


In [25]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       432
           1       0.83      0.85      0.84       123

    accuracy                           0.93       555
   macro avg       0.89      0.90      0.90       555
weighted avg       0.93      0.93      0.93       555
 



In [25]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.92      0.95       117
           1       0.72      0.92      0.81        25

    accuracy                           0.92       142
   macro avg       0.85      0.92      0.88       142
weighted avg       0.94      0.92      0.93       142
 



In [24]:
#distillbert_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [38]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.8009703180009733
